## 로컬 환경에서 PDF 검색하기 1단계 step3
- PDF 문서 여러개(2개) 로드하고 벡터 DB 생성
- PDF 문서를 로드하고 한국어 임베딩 모델을 적용하여 벡터 DB 생성
- 생성된 벡터 DB 에 검색기를 생성하고 LLM 모델을 추가하여 검색하기 

### - 사용한 임베딩 모델 jhgan/ko-sroberta-multitask
### - 사용한 LLM llama3.2

In [1]:
# 필요한 라이브러리 임포트
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings  
from sentence_transformers import SentenceTransformer

# 'data' 폴더 내의 모든 PDF 파일 로드
loader = DirectoryLoader(
    'data',
    glob='*.pdf',
    loader_cls=PyPDFLoader
)
documents = loader.load()

# Step 2: 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(documents)

In [2]:
# Step 3: SentenceTransformer 모델을 LangChain의 Embeddings 클래스로 감싸기
class KoSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        """문서 리스트를 벡터로 변환"""
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        """검색 쿼리를 벡터로 변환"""
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]


# Step 4: 모델 로드 및 FAISS 인덱스 생성
embedding_model = KoSentenceTransformerEmbeddings("jhgan/ko-sroberta-multitask")
faiss_index = FAISS.from_documents(split_documents, embedding_model)

In [3]:
# Step 5: FAISS 검색기 생성 (as_retriever() 사용)
retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [4]:
# 검색기 테스트
query = "삼성전자가 개발한 AI 이름이 뭐야?"
retriever.invoke(query)

[Document(id='684d881c-0a48-46f5-be92-45c5cefd2922', metadata={'source': 'data\\SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'page_label': '13'}, page_content='SPRi AI Brief |  2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 AI 모델 ‘삼성 가우스’를 공개n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents'),
 Document(id='b418a6c2-3cba-4472-b411-adbb40cb25c5', metadata={'source': 'data\\SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'page_label': '13'}, page_content='KEY Contents\n£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원n삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 ‘삼성 가우스’를 최초 공개∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 최적화된 크기의 모델 선택이 가능∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 단계적으로 탑재할 계획n삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 이미지 모델의 3개 모델로 

In [5]:
# Step 6 : 프롬프트 생성 
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean,and make sure the answer ends with '입니다'.

#Context: 
{context}

#Question:
{question}

#Answer(Ensure the response ends with '입니다'):"""
)

In [6]:
# Step 7 : 언어 모델 (LLM) 생성
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2")

C:\Users\morcs\AppData\Local\Temp\ipykernel_19252\2227855636.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


In [7]:
# 단계 8: 체인(Chain) 생성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "삼성전자가 개발한 AI 이름이 뭐야?"
response = chain.invoke(question)
print(response)

삼성전자가 개발한 AI 이름은 '삼성 가우스'입니다.


In [9]:
question = "포항에서 열리는 축제 이름이 뭐야?"
response = chain.invoke(question)
print(response)

청년 문화 페스티벌입니다.
